## Catalog access

### Create a client

In [ ]:
import pandas
try:
    from jupyterlabutils.notebook import get_catalog, retrieve_query
except ModuleNotFoundError:
    from rubin_jupyter_utils.lab.notebook import get_catalog, retrieve_query

pandas.set_option('display.max_rows', 1000)
service = get_catalog()

### Finding tables to query

In [ ]:
results = service.search("SELECT table_name, schema_name, description FROM TAP_SCHEMA.tables")
results.to_table().show_in_notebook()

### Pick a table and look at the schema

In [ ]:
results = service.search("SELECT column_name, description, unit FROM TAP_SCHEMA.columns WHERE table_name = 'wise_00.allwise_p3as_mep'")
results.to_table().show_in_notebook()

### Do a cone search

In [ ]:
results = service.search("SELECT w1mpro_ep, w2mpro_ep, w3mpro_ep FROM wise_00.allwise_p3as_mep WHERE CONTAINS(POINT('ICRS', ra, decl), CIRCLE('ICRS', 192.85, 27.13, .2)) = 1")
results.to_table()[0:50].show_in_notebook()

In [ ]:
len(results)

### Plot the result

In [ ]:
from bokeh.plotting import figure, output_file, show
from bokeh.models import ColumnDataSource, HoverTool
from bokeh.io import output_notebook
from bokeh.resources import INLINE
from rubin_jupyter_utils.lab import show_with_bokeh_server
import pandas
import numpy

output_notebook(INLINE, hide_banner=True)

dataframe = results.to_table().to_pandas()

dataframe['w1mw2'] = dataframe['w1mpro_ep'] - dataframe['w2mpro_ep']
dataframe['w2mw3'] = dataframe['w2mpro_ep'] - dataframe['w3mpro_ep']
null23_idx = pandas.isna(dataframe['w2mw3'])
null12_idx = pandas.isna(dataframe['w1mw2'])
null_idx = null23_idx | null12_idx

datasource = ColumnDataSource(dataframe[numpy.logical_not(null_idx)])
no_null_dataframe = dataframe[numpy.logical_not(null_idx)]

In [ ]:
# Make a plot of a certain size
p = figure(plot_width=800, plot_height=800)
p.yaxis.axis_label = '[3.6] - [4.6] (Profile fit magnitude)'
p.xaxis.axis_label = '[4.6] - [12] (Profile fit magnitude)'

# Add a circle renderer with a size, color, and alpha.
# x and y control which columns to use from the dataframe.
p.circle(x='w2mw3', y='w1mw2', source=datasource, size=5, color="navy", alpha=0.1)

# show the results
show_with_bokeh_server(p)

In [ ]:
# Make a plot of a certain size
p = figure(plot_width=800, plot_height=800)
p.yaxis.axis_label = '[3.6] - [4.6] (Profile fit magnitude)'
p.xaxis.axis_label = '[4.6] - [12] (Profile fit magnitude)'

# Bin data and add hover tool
r, bins = p.hexbin(no_null_dataframe['w2mw3'], no_null_dataframe['w1mw2'], size=0.03, hover_color='blue', hover_alpha=0.8)

p.add_tools(HoverTool(
    tooltips=[("count", "@c")],
    mode="mouse", point_policy="follow_mouse", renderers=[r]
))

# show the results
show_with_bokeh_server(p)